In [ ]:
##### the ideal is to pass a number-of-days-back variable to a function and have the api return every result from that date in the past to today
##### depending on our ability to filter dates in the api url, this could be done in 2 ways
########## if we can't use filter in api string
################# return (call of entire year(s) of crimes and filter them in python)
########## if we can use filter in api string
################# return (call api with date range in url, potentially 2 calls, one for each calendar year )


In [1]:
import pandas as pd
import requests
import json
import datetime

In [2]:
def _annualCall(year,daysback):
    #today get today's date
    today = datetime.datetime.now()
    #### crime api url
    crime_url =f"https://services.arcgis.com/afSMGVsC7QlRK1kZ/arcgis/rest/services/Police_Incidents_{year}/FeatureServer/0/query?where=reportedDateTime>=CURRENT_TIMESTAMP+-INTERVAL\'{daysback}\'DAY&outFields=reportedDateTime,offense,description,centerLong,centerLat&outSR=4326&f=json"
    ### make api request
    response = requests.get(crime_url).json()
    ## reduce to a variable that just includes the features
    features = [x for x in response['features']]

    return features

In [10]:
##### this function determines how many times we'll need to hit the API, handles it accordingly, and returns cleaned data
##### this function will accept daysback as a parameter and return all crimes from (today - daysback) through today
##### the way this is written, i think the daysback interger might need to be under a certain amount
def callCrimeAPI(daysback):

    # ### define the current  date and the date x number of days in the past(iefirst_day)
    today = datetime.datetime.now()
    first_day = today - datetime.timedelta(days=daysback)
    print(f'your requested date range is {first_day} through {today}')

    ##### this if statement only works if the first_day is in the current or prior calendar year. It won't work if the 'daysback' variable puts us more than 2 calendar years in the past
    ##### i could put a limit on the value of daysback with a try except block
    ##### ..... or i could make the code more dynamic by counting the number of years between 'today' and 'first_day', storing that as a variable, then doing that number of api calls

    #### if the begining date is in the same year and today's year...
    if (today.year == first_day.year):
        print('hit the current year api for the date range')
        #### just call the api once for this year 
        crime_list = _annualCall(today.year,daysback)
    else:
        print('hit the api twice, once for each year, and combine the results')
        #### get the datetime object for the begining of this year
        start_of_current_year = datetime.datetime(today.year,1,1)
        ### how many days have passed since the begining of the year
        elapsed_time_delta = today - start_of_current_year
        ### hit the api twice, once for each year
        crime_list = _annualCall(today.year,elapsed_time_delta.days)+_annualCall(first_day.year,daysback)

    clean_crime_list =[]

    ##### for all features in the current and prior year.....
    for feature in crime_list :
        clean_feature={
            'date':datetime.datetime.fromtimestamp(feature['attributes']['reportedDateTime']/1000).strftime("%m/%d/%Y"),
            'time':datetime.datetime.fromtimestamp(feature['attributes']['reportedDateTime']/1000).strftime("%H:%M:%S"),
            'centerLong': feature['attributes']['centerLong'],
            'centerLat':feature['attributes']['centerLat'],
            'description':feature['attributes']['description'].strip()
            }
        clean_crime_list.append(clean_feature)
    return clean_crime_list

In [17]:

##### this crimedata variable is THE variable we'll use 
crimedata=callCrimeAPI(400)

your requested date range is 2020-02-04 17:16:27.449821 through 2021-03-10 17:16:27.449821
hit the api twice, once for each year, and combine the results
elapsed time delta: 68 days, 17:16:27.449821 -- type:<class 'datetime.timedelta'>
elapsed time delta days: 68 -- type:<class 'int'>
first year crime length21993
current year crime length3428
combined year crime length25421


In [18]:
print(len(crimedata))

25421


In [21]:

#### turn all the dates from the api response into a list, converting each of the date strings to date time
datelist = [datetime.datetime.strptime(x['date'],'%m/%d/%Y') for x in crimedata]

#### finding the oldest and youngest dates
oldest = max(datelist)
youngest = min(datelist)
print(f'Returned dates range from {youngest.date()} to {oldest.date()}')


Returned dates range from 2020-02-04 to 2021-03-09
